In [ ]:
import os
import zipfile


In [ ]:
z=zipfile.ZipFile("/content/drive/MyDrive/ECG/archive.zip")
z.extractall()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization

In [ ]:
train_dir=r"/content/train"
test_dir=r"/content/test"

In [ ]:
def process(image,label):
  image=tf.cast(image,tf.float32)
  image=(image-127.5)/127.5
  return image,label

In [ ]:
# Load the training and validation datasets
train_ds = image_dataset_from_directory(
    directory=train_dir,
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(224, 224)
)

validation_ds = image_dataset_from_directory(
    directory=test_dir,
    labels="inferred",
    label_mode="int",
    batch_size=32,
    image_size=(224, 224)
)


Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [ ]:
train_ds=train_ds.map(process)
validation_ds=validation_ds.map(process)

In [ ]:
base_model=ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_2[0]… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 23,534,592 (89.78 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [ ]:
base_model.trainable = True


In [ ]:
model = Sequential()
model.add(base_model)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(7, activation="softmax"))


In [ ]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy", tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=50,
)

Epoch 1/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 399s 368ms/step - accuracy: 0.2929 - loss: 1.9179 - sparse_categorical_accuracy: 0.2929 - val_accuracy: 0.2804 - val_loss: 2.3222 - val_sparse_categorical_accuracy: 0.2804
Epoch 2/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 286s 318ms/step - accuracy: 0.4056 - loss: 1.5570 - sparse_categorical_accuracy: 0.4056 - val_accuracy: 0.5050 - val_loss: 1.3373 - val_sparse_categorical_accuracy: 0.5050
Epoch 3/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 322s 318ms/step - accuracy: 0.4877 - loss: 1.3374 - sparse_categorical_accuracy: 0.4877 - val_accuracy: 0.4709 - val_loss: 1.6558 - val_sparse_categorical_accuracy: 0.4709
Epoch 4/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 286s 319ms/step - accuracy: 0.5161 - loss: 1.2703 - sparse_categorical_accuracy: 0.5161 - val_accuracy: 0.4478 - val_loss: 1.4521 - val_sparse_categorical_accuracy: 0.4478
Epoch 5/50
898/898 ━━━━━━━━━━━━━━━━━━━━ 322s 318ms/step - accuracy: 0.5535 - loss: 1.1852 - sparse_categorical_accuracy: 0.5535 - val_accuracy: 0.5368 -

In [ ]:
model.save("model.h5")